## 21장 GPT의 기억력 향상: 맥락 유지를 위한 다양한 접근 방식

In [1]:
!pip install openai
!pip install httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [2]:
import os
from openai import OpenAI

with open("chatgpt.env") as env:
  for line in env:
    key, value = line.strip().split("=")
    os.environ[key] = value

client = OpenAI(api_key  = os.environ['API_KEY'])

## 21장 GPT의 기억력 향상: 맥락 유지를 위한 다양한 접근 방식

### 21-2. GPT의 맥락 상실: 대화를 잊어버리는 챗봇

In [3]:
import click

from openai import OpenAI

# OpenAI API 키 설정 및 초기화
def init_api():
    with open("chatgpt.env") as env:
       for line in env:
           key, value = line.strip().split("=")
           os.environ[key] = value

init_api()
client = OpenAI(api_key  = os.environ.get("API_KEY"))

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 프롬프트 설정
system_prompt = "당신은 도움이 되는 어시스턴트입니다."

# 기본 메시지 설정
base_messages = [
    {
        "role": "system",
        "content": system_prompt
    }
]

# 인터랙티브 세션 시작
while True:
    # 기본 메시지 복사
    messages = base_messages.copy()

    # 사용자 입력 받기
    request = input(
        click.style(
            "입력: (종료하려면 'exit' or ‘quit’ 입력): ",
            fg="green"  # 입력 메시지를 초록색으로 표시
        )
    )

    # 'exit' 또는 'quit' 입력 시 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 입력을 메시지에 추가
    messages.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API에 메시지 전송
    response = client.chat.completions.create(
        model=model,        # 모델 설정
        messages=messages,  # 메시지 리스트 전송
    )

    # 응답 내용 가져오기
    content = response.choices[0].message.content.strip()

    # 출력 결과를 보기 좋게 표시
    click.echo(
        click.style(
            "출력: ",
            fg="yellow"  # 출력 메시지를 노란색으로 표시
        ) + content
    )

    # 새로운 줄 추가
    click.echo()


입력: (종료하려면 'exit' or ‘quit’ 입력): 안녕하세요. 짧은 한줄로 답변을 해 주렴.
출력: 안녕하세요! 어떤 도움이 필요하신가요?

입력: (종료하려면 'exit' or ‘quit’ 입력): 제 고양이는 토토이고 검은색이예요.
출력: 토토라는 이름의 검은색 고양이는 정말 매력적일 것 같아요! 고양이는 성격도 다양하고 각자 독특한 개성을 가지고 있죠. 토토와 함께 하는 시간은 즐거운가요? 어떤 재미있는 에피소드가 있었는지도 궁금합니다!

입력: (종료하려면 'exit' or ‘quit’ 입력): 토토는 순하고, 매우 귀여워요.
출력: 토토가 순하고 귀엽군요! 어떤 점이 특히 귀엽다고 느끼시나요? 애완동물인가요, 아니면 어떤 캐릭터인가요? 더 이야기해 주시면 좋을 것 같아요!

입력: (종료하려면 'exit' or ‘quit’ 입력): 토토는 정말 말썽장이랍니다. 그러나 때때로 장난을 많이 쳐서 다치기도 해요.
출력: 토토가 장난을 많이 치는군요! 그런 모습이 귀엽기도 하지만, 다치지 않게 조심하는 게 중요하죠. 혹시 토토가 어떤 장난을 자주 치는지 궁금한데요, 어떤 일들이 있었는지 공유해 주실 수 있나요? 어떤 조치를 취하면 토토가 더 안전하게 놀 수 있을지도 함께 고민해 볼 수 있을 것 같아요.

입력: (종료하려면 'exit' or ‘quit’ 입력): 토토에 대해 궁금한게 있나요?
출력: 토토는 스포츠 경기의 결과를 예측하여 베팅하는 형태의 도박입니다. 다양한 스포츠 종목에 대해 선택할 수 있으며, 특정 경기의 승패, 점수 등의 요소를 맞추는 방식으로 진행됩니다. 토토에 대해 궁금한 점이 더 있으면 구체적으로 말씀해주시면 답변해드리겠습니다! 예를 들어, 규칙, 운영 방식, 종류, 안전한 베팅 방법 등 여러 가지 주제가 있을 수 있습니다.

입력: (종료하려면 'exit' or ‘quit’ 입력): 토토가 게임인가요?
출력: "토토가"는 특정 게임 이름이 아니라, "토토"와 "가"라는 두 개의 단어를 조합한 것일 수 있습니다. "토토"는 일

### 21-3. 대화 맥락 유지의 해법: GPT에 히스토리 추가하기

In [5]:
# app.py

import click
# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
# OpenAI API 키 설정 및 초기화
# init_api()
# client = OpenAI(api_key  = os.environ.get("API_KEY"))

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 프롬프트 설정
system_prompt = "당신은 도움이 되는 어시스턴트입니다."

# 초기 기록(history) 설정
history = [
    {
        "role": "system",
        "content": system_prompt
    }
]

# 인터랙티브 세션 시작
while True:
    # 사용자 입력 받기
    request = input(
        click.style(
            "입력: (종료하려면 'exit' or 'quit' 입력): ",
            fg="green"  # 입력 메시지를 초록색으로 표시
        )
    )

    # 'exit' 또는 'quit' 입력 시 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 메시지를 기록(history)에 추가
    history.append(
        {
            "role": "user",
            "content": f"{request}"
        }
    )

    # API에 메시지 전송
    response = client.chat.completions.create(
        model=model,  # 사용할 모델 설정
        messages=history,  # 전체 대화 기록(history) 전송
    )

    # 응답 내용 가져오기
    content = response.choices[0].message.content.strip()

    # 디버그: 현재까지의 대화 기록 출력
    click.echo(
        click.style(
            "히스토리: ",
            fg="blue"  # 기록 메시지를 파란색으로 표시
        ) + str(history)
    )

    # 출력 결과를 보기 좋게 표시
    click.echo(
        click.style(
            "출력: ",
            fg="yellow"  # 출력 메시지를 노란색으로 표시
        ) + content
    )

    # 모델의 응답을 기록(history)에 추가
    history.append(
        {
            "role": "assistant",
            "content": f"{content}"
        }
    )

    # 새로운 줄 추가
    click.echo()


입력: (종료하려면 'exit' or 'quit' 입력): 제 강아지는 토토이고 검은색이예요.
히스토리: [{'role': 'system', 'content': '당신은 도움이 되는 어시스턴트입니다.'}, {'role': 'user', 'content': '제 강아지는 토토이고 검은색이예요.'}]
출력: 토토는 귀여운 이름이네요! 검은색 강아지는 정말 멋진 외모를 가지고 있을 것 같아요. 토토와 함께하는 특별한 순간이나 좋아하는 놀이가 있나요?

입력: (종료하려면 'exit' or 'quit' 입력): 바닷가에 놀러가서 공놀이한 것이요.
히스토리: [{'role': 'system', 'content': '당신은 도움이 되는 어시스턴트입니다.'}, {'role': 'user', 'content': '제 강아지는 토토이고 검은색이예요.'}, {'role': 'assistant', 'content': '토토는 귀여운 이름이네요! 검은색 강아지는 정말 멋진 외모를 가지고 있을 것 같아요. 토토와 함께하는 특별한 순간이나 좋아하는 놀이가 있나요?'}, {'role': 'user', 'content': '바닷가에 놀러가서 공놀이한 것이요.'}]
출력: 바닷가에서 공놀이를 하다니 정말 즐거운 시간이었겠네요! 물튀기는 소리와 바람의 기분이 토토에게도 즐거움을 줄 것 같습니다. 바닷가에서 토토는 물에 들어가기도 했나요, 아니면 모래놀이를 더 좋아했나요?

입력: (종료하려면 'exit' or 'quit' 입력): 모래 놀이를 좋아한 것 같아요.
히스토리: [{'role': 'system', 'content': '당신은 도움이 되는 어시스턴트입니다.'}, {'role': 'user', 'content': '제 강아지는 토토이고 검은색이예요.'}, {'role': 'assistant', 'content': '토토는 귀여운 이름이네요! 검은색 강아지는 정말 멋진 외모를 가지고 있을 것 같아요. 토토와 함께하는 특별한 순간이나 좋아하는 놀이가 있나요?'}, {'role': 'use

### 21-4. 히스토리 관리의 문제: 대화 길이에 따른 토큰 제한과 비용

### 22-5. 효율적인 챗봇 대화 관리: 후입선출 방식으로 최신 맥락 유지하기

### The Problem with Last in First out Memory - p228

*	히스토리 파일이 저장됩니다.
*	사용자가 프롬프트를 입력하면, 프로그램은 가장 최근 N개의 프롬프트를 히스토리에서 읽고 사용자의 프롬프트와 함께 API로 전송합니다.
*	그런 다음 프로그램은 사용자의 프롬프트와 응답을 히스토리 파일에 저장합니다.
*	히스토리 파일은 최신 프롬프트와 응답으로 계속 업데이트됩니다.
*	프로그램은 이 과정을 반복합니다. 마지막 N개의 프롬프트를 읽고 사용자의 프롬프트와 함께 API로 전송하는 방식입니다.


In [6]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
# app.py

import click, json  # 클릭 및 JSON 모듈 불러오기
import os
import numpy as np

# 아래 불러오는 부분은 코드를 .py를 작성한 이후에 아래와 같이 불러올 수 있습니다.
# API 클라이언트 및 임베딩 함수
# from api import client, get_embedding

# 전처리 및 코사인 유사도 함수
# from utils import preprocess_text, cosine_similarity
# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
# 초기 API KEY 설정 및 OpenAI 초기화
# init_api()
# client = OpenAI(api_key  = os.environ.get("API_KEY"))

# 두 벡터 간의 코사인 유사도를 계산하는 함수
def cosine_similarity(a, b):
    numerator = np.dot(a, b)  # 벡터 내적 계산
    denominator = np.linalg.norm(a) * np.linalg.norm(b)  # 두 벡터의 크기 곱 계산
    return numerator / denominator  # 코사인 유사도 반환

# 텍스트 전처리 함수
def preprocess_text(text):
    from nltk.corpus import stopwords        # 불용어(stopwords) 모듈 임포트
    from nltk.stem import PorterStemmer       # 어간 추출(stemming)을 위한 PorterStemmer 임포트
    from nltk.tokenize import word_tokenize   # 단어 토크나이저(word_tokenize) 임포트

    # 텍스트를 단어 단위로 토큰화
    tokens = word_tokenize(text)

    # 모든 단어를 소문자로 변환
    tokens = [word.lower() for word in tokens]

    # 구두점(punctuation) 제거
    words = [word for word in tokens if word.isalpha()]

    # 불용어(stop words) 제거
    stop_words = set(stopwords.words('english'))  # 영어 불용어 집합 생성
    words = [word for word in words if word not in stop_words]

    # 어간 추출 (Stemming)
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]

    # 전처리된 단어들을 공백으로 연결하여 반환
    return ' '.join(stemmed_words)

def get_embedding(text, model):
    # 텍스트의 줄바꿈 문자를 공백으로 대체
    text = text.replace("\n", " ")
    # 임베딩 생성 후 반환
    return client.embeddings.create(
        input=[text],  # 입력 텍스트를 리스트 형식으로 전달
        model=model  # 사용할 모델 설정
    ).data[0].embedding

# 선택할 맥락의 개수 설정
context_window = 2

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 프롬프트 설정
system_prompt = "당신은 도움이 되는 어시스턴트입니다."
history_file_path = 'context.txt'  # 대화 기록 파일 경로 설정
full_history = []  # 전체 대화 기록 초기화
global_context = [
    {
        "role": "system",
        "content": system_prompt
    }
]

# 파일을 'w' 모드로 열고 즉시 닫아 초기화
with open(history_file_path, 'w') as file:
    pass

# 대화 기록을 파일에 저장하는 함수
def save_history_to_file(history):
    """
    대화 기록을 파일에 저장.
    """
    with open(history_file_path, "w") as f:
        f.write(json.dumps(history))  # 대화 기록을 JSON 문자열로 저장

# 파일에서 대화 기록을 불러오는 함수
def load_history_from_file():
    """
    파일에서 대화 기록을 불러오기.
    """
    with open(history_file_path, "r") as f:
        try:
            history = json.loads(f.read())  # 파일에서 JSON 형식의 기록 읽어오기
            return history  # 기록 반환
        except json.JSONDecodeError:
            return []  # 파일이 비어있거나 손상된 경우 빈 리스트 반환

# 대화 기록을 코사인 유사도 기반으로 정렬하는 함수
def sort_history(history, prompt, context_window):
    """
    대화 기록을 코사인 유사도를 기준으로 정렬.
    가장 유사한 context_window 개수만큼 반환.
    """
    sorted_history = []  # 정렬된 대화 기록을 저장할 리스트 초기화

    # 대화 기록의 각 세그먼트(항목)를 순회하며 유사도 계산
    for segment in history:
        content = segment['content']  # 현재 세그먼트의 'content' 값 가져오기
        preprocessed_content = preprocess_text(content)  # 세그먼트 내용 전처리
        preprocessed_prompt = preprocess_text(prompt)  # 사용자 프롬프트 전처리

        # 세그먼트와 프롬프트에 대한 임베딩 생성
        embedding_model = "text-embedding-ada-002"
        embedding_content = get_embedding(preprocessed_content, embedding_model)
        embedding_prompt = get_embedding(preprocessed_prompt, embedding_model)

        # 두 임베딩 간의 코사인 유사도 계산
        similarity = cosine_similarity(embedding_content, embedding_prompt)

        # 세그먼트와 유사도 값을 리스트에 추가
        sorted_history.append((segment, similarity))

    # 유사도 값을 기준으로 세그먼트 정렬 (내림차순)
    sorted_history = sorted(sorted_history, key=lambda x: x[1], reverse=True)

    # 정렬된 세그먼트에서 세그먼트만 추출하여 context_window 개수만큼 반환
    sorted_history = [x[0] for x in sorted_history]

    return sorted_history[:context_window]

# 인터랙티브 세션 시작
while True:
    # 사용자 입력 받기
    request = input(
        click.style(
            "입력: (종료하려면 'exit' 또는 'quit'): ",
            fg="green"  # 입력 메시지를 초록색으로 표시
        )
    )

    # 'exit' 또는 'quit' 입력 시 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 프롬프트를 기록(history)에 추가
    user_prompt = {
        "role": "user",
        "content": request
    }

    # 파일에서 대화 기록을 불러와 새로운 메시지 추가
    full_history = load_history_from_file()

    # 대화 기록을 정렬하여 가장 관련성 높은 기록 선택
    sorted_history = sort_history(full_history, request, context_window)
    sorted_history.append(user_prompt)  # 사용자 프롬프트 추가

    # 전체 기록에 글로벌 컨텍스트 추가
    messages = global_context + sorted_history

    # API에 메시지 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,  # 생성할 최대 토큰 수 설정
        temperature=1,  # 출력의 창의성 조절
    )

    # 디버그: 현재까지의 대화 기록 출력
    click.echo(
        click.style(
            "기록: ",
            fg="blue"  # 기록 메시지를 파란색으로 표시
        ) + str(json.dumps(messages, indent=4))
    )

    # 응답 내용 가져오기
    content = response.choices[0].message.content.strip()

    # 출력 결과를 보기 좋게 표시
    click.echo(
        click.style(
            "출력: ",
            fg="yellow"  # 출력 메시지를 노란색으로 표시
        ) + content
    )

    # 사용자 프롬프트를 전체 기록에 추가
    full_history.append(user_prompt)

    # 모델의 응답을 전체 기록에 추가
    full_history.append(
        {
            "role": "assistant",
            "content": content
        }
    )

    # 대화 기록을 파일에 저장
    save_history_to_file(full_history)

입력: (종료하려면 'exit' 또는 'quit'): 안녕하세요!
기록: [
    {
        "role": "system",
        "content": "\ub2f9\uc2e0\uc740 \ub3c4\uc6c0\uc774 \ub418\ub294 \uc5b4\uc2dc\uc2a4\ud134\ud2b8\uc785\ub2c8\ub2e4."
    },
    {
        "role": "user",
        "content": "\uc548\ub155\ud558\uc138\uc694!"
    }
]
출력: 안녕하세요! 어떻게 도와드릴까요?
입력: (종료하려면 'exit' 또는 'quit'): 나의 강아지는 슈나우저예용.
기록: [
    {
        "role": "system",
        "content": "\ub2f9\uc2e0\uc740 \ub3c4\uc6c0\uc774 \ub418\ub294 \uc5b4\uc2dc\uc2a4\ud134\ud2b8\uc785\ub2c8\ub2e4."
    },
    {
        "role": "user",
        "content": "\uc548\ub155\ud558\uc138\uc694!"
    },
    {
        "role": "assistant",
        "content": "\uc548\ub155\ud558\uc138\uc694! \uc5b4\ub5bb\uac8c \ub3c4\uc640\ub4dc\ub9b4\uae4c\uc694?"
    },
    {
        "role": "user",
        "content": "\ub098\uc758 \uac15\uc544\uc9c0\ub294 \uc288\ub098\uc6b0\uc800\uc608\uc6a9."
    }
]
출력: 슈나우저는 아주 귀여운 강아지죠! 그들은 지능이 좋고, 성격도 활발하며 충성심이 강한 편입니다. 슈나우저와 함께 즐길 수 있는 특별한 활동이나 궁금한 점이 있

### 21-6. 후입선출(LIFO) 메모리의 한계점

챗봇이 대화를 기억하는 방식 중 하나인 '후입선출' 방식에는 중요한 문제점이 있습니다. 이 방식은 가장 최근의 대화만 기억하고 오래된 대화는 잊어버리는데, 복잡한 대화를 나눌 때 이것이 큰 문제가 될 수 있습니다. 사용자가 이전에 했던 중요한 이야기를 챗봇이 기억하지 못하면, 대화가 자연스럽게 이어지지 않아 사용자가 답답함을 느낄 수 있습니다.

### 21-7 최적의 대화 맥락을 선택하는 방법

In [ ]:
# app.py

import click, json  # 클릭 및 JSON 모듈 불러오기
import os, numpy
# 아래 불러오는 부분은 로컬에서 수행할 때, .py를 만들고 불러올 수 있습니다.
# API 클라이언트 및 임베딩 함수
# from api import client, get_embedding
# 전처리 및 코사인 유사도 함수
# from utils import preprocess_text, cosine_similarity
# 실행 환경이 구글 코랩의 경우, 인증 부분은 앞에서 실행했다면, 작성하지 않아도 됩니다.
## 초기 인증 함수 불러오기
## init_api()

client = OpenAI(api_key  = os.environ.get("API_KEY"))

# 두 벡터 간의 코사인 유사도를 계산하는 함수
def cosine_similarity(a, b):
    numerator = np.dot(a, b)  # 벡터 내적 계산
    denominator = np.linalg.norm(a) * np.linalg.norm(b)  # 두 벡터의 크기 곱 계산
    return numerator / denominator  # 코사인 유사도 반환

# 텍스트 전처리 함수
def preprocess_text(text):
    from nltk.corpus import stopwords        # 불용어(stopwords) 모듈 임포트
    from nltk.stem import PorterStemmer       # 어간 추출(stemming)을 위한 PorterStemmer 임포트
    from nltk.tokenize import word_tokenize   # 단어 토크나이저(word_tokenize) 임포트

    # 텍스트를 단어 단위로 토큰화
    tokens = word_tokenize(text)

    # 모든 단어를 소문자로 변환
    tokens = [word.lower() for word in tokens]

    # 구두점(punctuation) 제거
    words = [word for word in tokens if word.isalpha()]

    # 불용어(stop words) 제거
    stop_words = set(stopwords.words('english'))  # 영어 불용어 집합 생성
    words = [word for word in words if word not in stop_words]

    # 어간 추출 (Stemming)
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(word) for word in words]

    # 전처리된 단어들을 공백으로 연결하여 반환
    return ' '.join(stemmed_words)

def get_embedding(text, model):
    # 텍스트의 줄바꿈 문자를 공백으로 대체
    text = text.replace("\n", " ")
    # 임베딩 생성 후 반환
    return client.embeddings.create(
        input=[text],  # 입력 텍스트를 리스트 형식으로 전달
        model=model  # 사용할 모델 설정
    ).data[0].embedding

# 선택할 맥락의 개수 설정
context_window = 2

# 사용할 모델 설정
model = "gpt-4o-mini"

# 시스템 프롬프트 설정
system_prompt = "You're a helpful assistant."
history_file_path = 'context.txt'  # 대화 기록 파일 경로 설정
full_history = []  # 전체 대화 기록 초기화
global_context = [
    {
        "role": "system",
        "content": system_prompt
    }
]

# 파일을 'w' 모드로 열고 즉시 닫아 초기화
with open(history_file_path, 'w') as file:
    pass

# 대화 기록을 파일에 저장하는 함수
def save_history_to_file(history):
    """
    대화 기록을 파일에 저장.
    """
    with open(history_file_path, "w") as f:
        f.write(json.dumps(history))  # 대화 기록을 JSON 문자열로 저장

# 파일에서 대화 기록을 불러오는 함수
def load_history_from_file():
    """
    파일에서 대화 기록을 불러오기.
    """
    with open(history_file_path, "r") as f:
        try:
            history = json.loads(f.read())  # 파일에서 JSON 형식의 기록 읽어오기
            return history  # 기록 반환
        except json.JSONDecodeError:
            return []  # 파일이 비어있거나 손상된 경우 빈 리스트 반환

# 대화 기록을 코사인 유사도 기반으로 정렬하는 함수
def sort_history(history, prompt, context_window):
    """
    대화 기록을 코사인 유사도를 기준으로 정렬.
    가장 유사한 context_window 개수만큼 반환.
    """
    sorted_history = []  # 정렬된 대화 기록을 저장할 리스트 초기화

    # 대화 기록의 각 세그먼트(항목)를 순회하며 유사도 계산
    for segment in history:
        content = segment['content']  # 현재 세그먼트의 'content' 값 가져오기
        preprocessed_content = preprocess_text(content)  # 세그먼트 내용 전처리
        preprocessed_prompt = preprocess_text(prompt)  # 사용자 프롬프트 전처리

        # 세그먼트와 프롬프트에 대한 임베딩 생성
        embedding_model = "text-embedding-ada-002"
        embedding_content = get_embedding(preprocessed_content, embedding_model)
        embedding_prompt = get_embedding(preprocessed_prompt, embedding_model)

        # 두 임베딩 간의 코사인 유사도 계산
        similarity = cosine_similarity(embedding_content, embedding_prompt)

        # 세그먼트와 유사도 값을 리스트에 추가
        sorted_history.append((segment, similarity))

    # 유사도 값을 기준으로 세그먼트 정렬 (내림차순)
    sorted_history = sorted(sorted_history, key=lambda x: x[1], reverse=True)

    # 정렬된 세그먼트에서 세그먼트만 추출하여 context_window 개수만큼 반환
    sorted_history = [x[0] for x in sorted_history]

    return sorted_history[:context_window]

# 인터랙티브 세션 시작
while True:
    # 사용자 입력 받기
    request = input(
        click.style(
            "입력: (종료하려면 'exit' 또는 'quit'): ",
            fg="green"  # 입력 메시지를 초록색으로 표시
        )
    )

    # 'exit' 또는 'quit' 입력 시 종료
    if request.lower() in ["exit", "quit"]:
        break

    # 사용자 프롬프트를 기록(history)에 추가
    user_prompt = {
        "role": "user",
        "content": request
    }

    # 파일에서 대화 기록을 불러와 새로운 메시지 추가
    full_history = load_history_from_file()

    # 대화 기록을 정렬하여 가장 관련성 높은 기록 선택
    sorted_history = sort_history(full_history, request, context_window)
    sorted_history.append(user_prompt)  # 사용자 프롬프트 추가

    # 전체 기록에 글로벌 컨텍스트 추가
    messages = global_context + sorted_history

    # API에 메시지 전송
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=200,  # 생성할 최대 토큰 수 설정
        temperature=1,  # 출력의 창의성 조절
    )

    # 디버그: 현재까지의 대화 기록 출력
    click.echo(
        click.style(
            "기록: ",
            fg="blue"  # 기록 메시지를 파란색으로 표시
        ) + str(json.dumps(messages, indent=4))
    )

    # 응답 내용 가져오기
    content = response.choices[0].message.content.strip()

    # 출력 결과를 보기 좋게 표시
    click.echo(
        click.style(
            "출력: ",
            fg="yellow"  # 출력 메시지를 노란색으로 표시
        ) + content
    )

    # 사용자 프롬프트를 전체 기록에 추가
    full_history.append(user_prompt)

    # 모델의 응답을 전체 기록에 추가
    full_history.append(
        {
            "role": "assistant",
            "content": content
        }
    )

    # 대화 기록을 파일에 저장
    save_history_to_file(full_history)


입력: (종료하려면 'exit' 또는 'quit'): I have a black cat.
기록: [
    {
        "role": "system",
        "content": "You're a helpful assistant."
    },
    {
        "role": "user",
        "content": "I have a black cat."
    }
]
출력: That's wonderful! Black cats can be so beautiful and have their own unique personalities. What's your cat's name, and what do you enjoy doing together?
입력: (종료하려면 'exit' 또는 'quit'): I have a white shirt.
기록: [
    {
        "role": "system",
        "content": "You're a helpful assistant."
    },
    {
        "role": "user",
        "content": "I have a black cat."
    },
    {
        "role": "assistant",
        "content": "That's wonderful! Black cats can be so beautiful and have their own unique personalities. What's your cat's name, and what do you enjoy doing together?"
    },
    {
        "role": "user",
        "content": "I have a white shirt."
    }
]
출력: A white shirt is a classic wardrobe staple! It can be dressed up or down and goes well with many 